In [1]:
# Importing required packages
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import re
import time
import datetime
from datetime import datetime
import socket
import getpass
import html2text as h2h
from html.parser import HTMLParser

#required packages
import mysql.connector as connection
from functools import reduce
import json
import os

In [2]:
# Load credentials from the JSON file
with open('db_credentials_trg.json') as f:
    credentials = json.load(f)

# Extract the credentials
p_host = credentials['host']
p_user = credentials['user']
p_password = credentials['password']
p_database = credentials['database']

In [7]:
st = []
ed = []
du = []
date_range= []
v_host_name = []

pd.set_option('display.max_column', None)

start = datetime.now()
st.append(start)

##Final Function
def generate_village_level_report(p_start_date, p_end_date):
    
    try:
        v_visit_started_date = p_start_date
        v_visit_ended_date = p_end_date
        
        def f_connect_to_database(p_host, p_user, p_password, p_database):
            while True:
                try:
                    # Setup database connection
                    v_mydb = connection.connect(
                        host=p_host,
                        user=p_user,
                        password=p_password,
                        database=p_database,
                        use_pure=True
                    )
                    v_cursor = v_mydb.cursor()
                    # print("Connected to the database\n")
                    return v_mydb, v_cursor

                except connection.Error as err:
                    print(f"Error: {err}")
                    print("\n retry !")
                    return None, None


        # Function to close the database connection
        def f_close_database_connection(p_mydb, p_cursor):
            try:
                if p_cursor is not None:
                    p_cursor.close()
                    # print("Cursor closed.")
                if p_mydb.is_connected():
                    p_mydb.close()
                    # print("Database connection closed.\n")
            except Exception as e:
                return f"An error occurred while closing the database connection: {e}"

        while True:
            v_mydb,v_cursor = f_connect_to_database(p_host, p_user, p_password, p_database)

            v_village = input("Enter Village Id: ")
            v_state = input("Enter State Id: ")
            v_district = input("Enter District Id: ")
            v_sanch = input("Enter Sanch Id: ")
            # v_visit_speciality = input("Enter the visit_speciality value: ")

            # Update variables based on input
            s_loc_village = 'and l.location_id = ' + v_village if v_village else 'and 1=1'
            s_loc_state = 'and lst.location_id = ' + v_state if v_state else 'and 1=1'
            s_loc_district = 'and ld.location_id = ' + v_district if v_district else 'and 1=1'
            s_loc_sanch = 'and ls.location_id = ' + v_sanch if v_sanch else 'and 1=1'

            if v_mydb is not None:

                #Close the database connection
                f_close_database_connection(v_mydb, v_cursor)
                break
          # Query with location filters
        v_query_concat = {'Query1': '''select pat.patient_id AS Patient_id, 
                                        l.name as Village,
                                        ls.name as Sanch,
                                        ld.name as District,
                                        l.state_province as State
                                from patient pat   
                                join patient_identifier pi on (pi.patient_id = pat.patient_id)
                                join location l on (pi.voided = 0 and pi.location_id = l.location_id)
                                join location_tag_map lmap on (lmap.location_id = l.location_id and lmap. location_tag_id = 8)
                                left join location ls on (ls.location_id = l.parent_location)
                                left join location ld on (ld.location_id = ls.parent_location)
                                LEFT JOIN location lst on (lst.location_id = ld.parent_location)
                                where pat.voided = 0 and pat.date_created between %s and %s''',
                         'Query2':s_loc_village+' ',
                         'Query3':s_loc_state+' ',
                         'Query4':s_loc_district+' ',
                         'Query5':s_loc_sanch+' '} 

        f_query = ''
        for query_name, query in v_query_concat.items():
            f_query = f_query + ' '+ query
        
        # print(f_query) 

        # SQL queries
        v_queries = {'Query1': f_query,
                     
                     'Query2':'''select pat.patient_id AS Patient_id,
                                        pat.creator,
                                        u.username as Sevika_Name,
                                        pa.value as hh_id
                                from    patient pat,
                                        users u,
                                        patient_identifier pi,
                                        person_attribute pa 
                                where   pat.voided = 0
                                and     u.user_id = pat.creator
                                and     pi.patient_id = pat.patient_id 
                                and     pi.voided = 0 
                                and     pa.voided = 0 
                                and     pa.person_attribute_type_id = 24 
                                and     pa.person_id = pat.patient_id
                                and     pat.date_created between %s and %s;''',
    
                     'Query3':'''SELECT pat.patient_id AS Patient_id,
                                    (la.value_reference) as Village_Type
                                    FROM patient pat 
                                    left join patient_identifier pi on pat.patient_id = pi.patient_id
                                    LEFT JOIN location l ON (pi.location_id = l.location_id)
                                    LEFT JOIN location_attribute la ON (la.location_id = l.location_id)
                                    WHERE pat.voided = 0
                                    and pi.voided = 0
                                    AND la.voided = 0
                                    AND la.attribute_type_id = 11
                                    AND pat.date_created between %s and %s
                                    GROUP BY pat.patient_id, la.value_reference;'''}
        
        def f_get_date_range_from_tables(p_mydb, p_cursor):
                    try:
                        if p_mydb.is_connected():
                            # SQL query to fetch the minimum and maximum dates from "visit" table
                            date_query = "SELECT MIN(date_created) AS min_date, MAX(date_created) AS max_date FROM patient WHERE voided = 0;"
        
                            p_cursor.execute(date_query)
                            v_date_results = p_cursor.fetchall()
        
                            if v_date_results:
                                v_min_date, v_max_date = v_date_results[0]
                                return v_min_date, v_max_date
        
                            else:
                                print("No data available in the tables.")
                                return None, None
                        else:
                            print("Database connection is not opened for getting the Minnimum and Maximum dates.")
                            return None, None
                    except Exception as e:
                        print(f"An error occurred: {e}")

        
        # Connect to the database
        v_mydb, v_cursor = f_connect_to_database(p_host, p_user, p_password, p_database)

        # Get the date range from the SQL tables
        v_min_table_date, v_max_table_date = f_get_date_range_from_tables(v_mydb, v_cursor)

        # Close the database connection
        f_close_database_connection(v_mydb, v_cursor)

        # Print the date range to inform the user
        print(f"Date range available in the SQL table: {v_min_table_date} to {v_max_table_date}\n")
        
        def f_execute_query_with_params(p_mydb, p_cursor, p_query, p_params):
            try:
                if p_mydb.is_connected():
                    p_cursor.execute(p_query, p_params)
                    v_results = p_cursor.fetchall()
                    v_columns = [desc[0] for desc in p_cursor.description]
                    df = pd.DataFrame(v_results, columns=v_columns)
                    # print(df)
                    return df
                else:
                    print("Database connection is not opened.")
                    return None  # Return an empty DataFrame if no results
            except Exception as e:
                print(f"An error occurred: {e}")
                return None
        
        # Modify the f_fetch_data1 function
        def f_fetch_data1(p_query, p_params):
            v_dataframe = []
            try:
                v_params = p_params
        
                # Connect to the database and get the cursor
                v_mydb, v_cursor = f_connect_to_database(p_host, p_user, p_password, p_database)
        
                # Execute each query with input parameters and store the output in a list
                for query_name, query in v_queries.items():
                        df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                        if df is not None:
                            v_dataframe.append(df)
                            # print(f"{query_name} executed successfully.\n")
        
                # Close the database connection
                f_close_database_connection(v_mydb, v_cursor)
        
                # Merge the DataFrames
                if len(v_dataframe) == 1:
                    return v_dataframe[0]
                elif len(v_dataframe) == 2:
                    # Merge DataFrames from the list
                    v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='Patient_id', how ='inner')
                    # print(v_data)
                    return v_data
                    
                elif len(v_dataframe) == 3:
                    # Check if the third DataFrame is empty
                    if v_dataframe[2].empty:
                        # Merge the first two DataFrames
                        v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='Patient_id', how ='inner')
                        
                         # Add an empty column 'village_type'
                        v_data['Village_Type'] = None  # or use np.nan or ''
                        # print(f'empty_dataframe_merged',v_data)
                        return v_data
                    else:
                        # Merge all three DataFrames
                        v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='Patient_id', how ='inner')
                        v_data = pd.merge(v_data, v_dataframe[2], on='Patient_id', how='left')
                        # print(f'final merged data',v_data)
                        return v_data
        
            except Exception as e:
                print('In this function', e)

        # Funciton for filtering the data between two days
        def f_filter_data(p_start_date, p_end_date, p_query):
            v_queries = []

            v_start_date1 = p_start_date
            v_end_date1 = p_end_date

            for i in range(len(p_query)):
                v_dummy = p_query[i].format(v_start_date1, v_end_date1)
                v_queries.append(v_dummy)

            v_data = f_fetch_data1(v_queries)
            # print(v_data)
            return v_data

        ## SQL query to fetch the minimum and maximum dates from "visit" table
        v_min_date = v_min_table_date
        v_last_date = v_max_table_date
        v_hours = v_min_date.hour
        v_minutes = v_min_date.minute
        v_seconds = v_min_date.second
    
        # print(f'Date Between **Starting Date - ({v_min_date}) and **End Date - ({v_last_date}).\n\n**The format is yyy-mm-dd\n')
        try:
            while True:
                v_strt_date_input = v_visit_started_date

                pd.to_datetime(v_strt_date_input, format='%Y-%m-%d')
                #v_start_date = v_strt_date_input+' '+str(v_hours)+':'+str(v_minutes)+':'+str(v_seconds)
                v_start_date = v_strt_date_input+' 00:00:00'
                date_format = '%Y-%m-%d %H:%M:%S'

                # Convert the string to a datetime object
                datetime_object = datetime.strptime(v_start_date, date_format)
                # print("\nDate Entered: \t",datetime_object)

                try:
                    if (datetime_object<=v_last_date): # (datetime_object>=v_min_date) and 
                            while True:
                                # Getting input from user
                                v_end_date_input = v_visit_ended_date
                                pd.to_datetime(v_end_date_input, format='%Y-%m-%d')
                                v_end_date = v_end_date_input+' 23:59:59'
    
                                #v_end_date = v_end_date+' 00:00:00'
                                # Convert the string to a datetime object
                                datetime_object1 = datetime.strptime(v_end_date, date_format)
                                # print("\nDate Entered: \t",datetime_object1,'\n')

                                try:
                                    if (datetime_object1<v_last_date) or (datetime_object1>datetime_object):
                                        if datetime_object1 < datetime_object:
                                            print('Incorrct Date. Please Enter the Date within the range.')
                                        else:
                                            # Getting the username
                                            v_hostname = socket.gethostname()
                                            v_host_name.append(v_hostname)
                        
                                            # Execute each query with input parameters
                                            v_params = (v_start_date, v_end_date)
                                                                                    
                                            # Entered data ranges
                                            date_range.append(v_start_date+' / '+v_end_date)
                                            
                                            # Passing the startdate, end date and query in the below function to fetch the data
                                            v_sample = f_fetch_data1(v_queries, v_params)
                                            
                                            v_sample = v_sample.drop_duplicates() # remove duplicates
                        
                                            # Fill missing values
                                            v_sample.fillna('', inplace=True)
                        
                                            # Aggregate the data
                                            aggregated_data = v_sample.groupby(['Sevika_Name', 'State', 'District', 'Sanch', 'Village', 'Village_Type'], as_index=False).agg({'Patient_id': 'count', 'hh_id': 'nunique'})
                                            # print(aggregated_data)
                                            
                                            # Rename the columns
                                            aggregated_data = aggregated_data.rename(columns={'Patient_id': 'Patient_count', 'hh_id': 'HouseHold_count'})
                                            # print(aggregated_data)
                                            
                                            sorted_data = aggregated_data.sort_values(by=['Sevika_Name', 'State', 'District', 'Sanch', 'Village', 'Village_Type'])
                                            # print(sorted_data)
                                            # print('\nVillage level report between 2 dates dataframe created\n')
                                            
                                            # Get the current date and time
                                            file_name = datetime.now()
                                            # Format the datetime object
                                            formatted_file_name = file_name.strftime("%Y-%m-%d_%H-%M-%S")
                        
                                            # Conver the dataframe into an excel file
                                            sorted_data.to_excel(f'village_level_report_{formatted_file_name}.xlsx', index=False)
                                            
                                            print(f'\nVillage_level_report_2_dates_{formatted_file_name}.xlsx Excel Output Created...:)\n')
                                            
                                            end = datetime.now()
                                            ed.append(end)
                        
                                            # duration = ed[-1]-st[-1]
                                            # du.append(duration)

                                            if ed[-1] is not None and st[-1] is not None:
                                                duration = ed[-1] - st[-1]
                                                du.append(duration)
                                            else:
                                                du.append(None)
                        
                                            # Find the length of the longest list
                                            max_length = max(len(v_host_name), len(date_range), len(st), len(ed), len(du))
                                            
                                            # Extend the shorter lists with None values
                                            v_host_name.extend([None] * (max_length - len(v_host_name)))
                                            date_range.extend([None] * (max_length - len(date_range)))
                                            st.extend([None] * (max_length - len(st)))
                                            ed.extend([None] * (max_length - len(ed)))
                                            du.extend([None] * (max_length - len(du)))
                        
                                            dur = {'Host_name':v_host_name, 'Date_Range':date_range, 'Start': st, 'End':ed, 'Duration':du}
                                            duration_df = pd.DataFrame(dur)
                        
                                            # Convert the 'Duration' column to seconds
                                            duration_df['Duration'] = duration_df['Duration'].dt.total_seconds()
                                            duration_df.to_excel('Duration2.xlsx', index=False)
                                            return duration_df

                                    else:
                                        print('Enter the date with range')
                                except Exception as e:
                                    print('Error 2', e)
                                    break
                        # break
                    else:
                        print('Error3')
                    break
                except Exception as e:
                    print(e)
        except Exception as e:
            print('Error 4', e)
    except Exception as e:
        print('Error in the main function: ', e)

In [10]:
generate_village_level_report('2023-05-01' , '2024-07-17')

Enter Village Id:  1010
Enter State Id:  51
Enter District Id:  876
Enter Sanch Id:  1007


Date range available in the SQL table: 2023-05-12 13:53:53 to 2024-07-17 14:36:21


Village_level_report_2_dates_2024-07-17_17-55-04.xlsx Excel Output Created...:)



,Host_name,Date_Range,Start,End,Duration
0,SindhuNaveenVaagmi,2023-05-01 00:00:00 / 2024-07-17 23:59:59,2024-07-17 17:48:37.811075,2024-07-17 17:49:04.969523,27.158448
1,SindhuNaveenVaagmi,2023-05-01 00:00:00 / 2024-07-17 23:59:59,NaT,2024-07-17 17:51:57.317264,199.506189
2,SindhuNaveenVaagmi,2023-05-01 00:00:00 / 2024-07-17 23:59:59,NaT,2024-07-17 17:55:04.306050,NaN
